*These aren't the droids you're looking for...*   
(just some code to make this entire thing more user-friendly, don't worry about it)

In [1]:
# use most of the browser size, don't do only the narrow thing in the middle
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

# avoid being flooded by log messages. Warnings are enough (let's hope we don't have any)
import logging
logging.basicConfig(level=logging.WARNING)

# ML4CREST: A Watering System with Machine Learning Calibration

We will use the machine learning calibration and add the regression information to the simulator.

But first, we will define the water tank and pump entities.

In [2]:
# import all CREST model concepts
from src.model import *

# import the CREST simulator
from src.simulator.simulator import Simulator

# import a library that can visualise our systems (using ELKjs)
from src.ui import elk

In [3]:
# define the resources within our system. We do it as static objects in a class so we have them all bundled together.
class Resources(object):
    on_off = Resource("switch", ["on", "off"])
    flow = Resource("L/sec", REAL)
    water = Resource("Liters", REAL)

In [4]:
class Pump(Entity):
    """
    A pump that is placed in an assumingly infinite source of water, such as a very big water tank, lake or similar. 
    We therefore don't model that the pump's inflow can vary. In fact the pump only has an outflow.
    
    For this example we also disregard the electricity that is required for the pump to operate, as we're only interested in modelling the water and spillage.
    """
    switch = Input(resource=Resources.on_off, value="off")
    
    rate = Local(resource=Resources.flow, value=0)  # the potential outflow of the pump
    output = Output(resource=Resources.flow, value=0)  # the actual output of the pump
    
    """ States """
    on = State()
    off = current = State()
    
    """ Transitions """
    off_to_on = Transition(source=off, target=on, guard=(lambda self: self.switch.value == "on"))
    on_to_off = Transition(source=on, target=off, guard=(lambda self: self.switch.value == "off"))
    
    """ Updates """
    @update(state=on, target=output)
    def set_on(self, dt=0):
        return self.rate.value

    @update(state=off, target=output)
    def set_off(self, dt=0):
        return 0

# plot a pump
elk.plot(Pump())

In [5]:
class Tank(Entity):
    """
    A tank is a very simple system. It has one inflow and one outflow. Inflow is defined by the water source, i.e. it's an input. 
    Outflow is defined by the tank itself. In our case it depends on size of the hose leaving the tank (static) and the current volume (more volume, more output due to pressure).
    We assume the outflow is piecewise linear, for simplicity. We could also try something more complex, but that's overkill...
    
    Additionally to inflow and outflow we will also measure how much water left the tank in total (quantity) 
    and how much water was spilt because we filled the volume higher than the capacity (spill).
    """
    inflow = Input(resource=Resources.flow, value=0)  # the current inflow
    outflow = Output(resource=Resources.flow, value=0)  # the current outflow
    spill = Output(resource=Resources.water, value=0)  # the amount of water spilt
    quantity = Output(resource=Resources.water, value=0)  # the volume of water that left the tank
    
    volume = Local(resource=Resources.water, value=0)  # the currently held water
    capacity = Local(resource=Resources.water, value=1)  # the maximum capacity, if we fill more, the rest is spilt

    limit1 = Local(resource=Resources.water, value=0)  # the limit between the different outflow rates
    limit2 = Local(resource=Resources.water, value=0)  # the limit between the different outflow rates
    rate0 = Local(resource=Resources.flow, value=0) # the rate of the lowest outflow
    rate1 = Local(resource=Resources.flow, value=0) # the rate of the middle outflow
    rate2 = Local(resource=Resources.flow, value=0) # the rate of the highest outflow
    
    """ States """
    full = State()
    partially = State()
    empty = current = State()
        
    """ Transitions """
    @transition(source=partially, target=full)
    def to_full(self):
        return self.volume.value >= self.capacity.value
    
    @transition(source=partially, target=empty)
    def to_empty(self):
        return self.volume.value <= 0
    
    @transition(source=[empty,full], target=partially)
    def to_partially(self):
        return 0 < self.volume.value and self.volume.value < self.capacity.value
    
    """ Updates """
    @update(state=[empty,partially,full], target=volume)
    def set_volume(self, dt):
        theoretical = 0
        if self.inflow.value > 0:
            theoretical = self.volume.value + self.inflow.value * dt
            if theoretical >= self.capacity.value:
                return self.capacity.value
            else:
                return theoretical
        else:
            theoretical = self.volume.value + self.outflow.pre * -1 * dt
            if theoretical >= self.capacity.value:
                return self.capacity.value
            else:
                return theoretical

    @update(state=[partially,full], target=outflow)
    def set_outflow_nonempty(self, dt):
        if self.volume.value > self.limit2.value:
            return self.rate2.value
        elif self.volume.value > self.limit1.value:
            return self.rate1.value
        else:
            return self.rate0.value
    
    @update(state=empty, target=outflow)
    def set_outflow_empty(self, dt):
        if self.inflow.value > self.rate0.value:
            return self.rate0.value
        else:
            return self.inflow.value
    
    @update(state=[empty,partially,full], target=quantity)
    def increase_quantity(self, dt):
        return self.quantity.value + self.outflow.value * dt
    
    
    @update(state=full, target=spill)
    def increase_spill(self, dt):
        netflow = self.inflow.value - self.outflow.value
        if netflow > 0:
            return self.spill.value + netflow * dt
        else:
            return self.spill.value

# plot a tank
elk.plot(Tank())  

# Machine learning calibration:
We define a WateringSystem. In the class' constructor (`__init__`) we access the functions that perform the machine learning regressions and use the return values to calibrate the system.


In [6]:
class WateringSystem(Entity):
    """
    The system is very simple. One pump, one tank. Hook up the connections.
    We connect the system's switch-input to the pump, the pump to the tank, and the tank's outputs to the system's outputs.
    Done. Check out the plot below for more info.
    """
    
    """ Ports """
    switch = Input(resource=Resources.on_off, value="off")  # turn the pump on / off
    output = Output(resource=Resources.flow, value=0)  # the current outflow of the system
    spill = Output(resource=Resources.water, value=0)  # the amount of water spilt
    quantity = Output(resource=Resources.water, value=0)  # the water that was poured over he plants
    
    """ Subentities """
    pump = Pump()
    tank = Tank()
    
    """ State (no transitions) """
    state = current = State()
    
    """ Influences """
    connect_switch = Influence(source=switch, target=pump.switch)
    connect_pump_to_tank = Influence(source=pump.output, target=tank.inflow)
    connect_output = Influence(source=tank.outflow, target=output)
    connect_spill = Influence(source=tank.spill, target=spill)
    connect_quantity = Influence(source=tank.quantity, target=quantity)
    
    # this one is hardcoded for three outflow rates and one inflow rate
    # but theoretically we could do it more dynamically (using CREST's add function)
    # but it's a bit too much for the ML introduction and not the point of this tutorial
    def __init__(self):
        import utils  # this is where the ML functions are
        # the the piecewise linear regressions, we want three parts ()
        outflow_ranges = utils.get_outflow("files/time_volume_right.csv", 3)
        assert len(outflow_ranges) == 3, "The number of outflow ranges is not three !"

        
        # outflow ranges are specified with (High, Low, Rate)
        # sort them, then the middle one tell us the volume ranges
        sorted_ranges = sorted(outflow_ranges, key=(lambda v: v[1]))
        self.tank.limit1.value = sorted_ranges[1][1]
        self.tank.limit2.value = sorted_ranges[1][0]
        
        # set the outflow-rates
        self.tank.rate0.value = abs(sorted_ranges[0][2])  # abs is necessary to assert that the value is positive
        self.tank.rate1.value = abs(sorted_ranges[1][2])
        self.tank.rate2.value = abs(0.0019)
        
        
        # inflow returns only one anyway
        inflow = utils.get_inflow("files/time_volume_right.csv")
        self.pump.rate.value = inflow[2]  # it was defined as (high, low, rate), we want rate

# elk.plot(WateringSystem())

# Simulation
We define a CREST simulator and use a `WateringSystem` entity as model.

Then we run the simulator (`stabilise` asserts the system is in a stable state.
We then turn the pump on (resp. off) and advance the specified time in that state.

Note, that for this initial scenario we chose the timings so that there is no spillage (we like our equipment and plants).  
We will perform a more thorough test once we have a more water-proof setup (and office).

In [7]:
sim = Simulator(WateringSystem())
sim.stabilise()

"""
This is the list of simulation phases. 
We define the value of the switch to set, and the time to advance with that setting.
We iterate over this list and then we're done. 
"""
evolution = [
    ("on", 10),
    ("off", 30),
    ("on", 10),
    ("off", 30),
    ("on", 10),
    ("off", 30)
]

# set values and advance according to evolution list
for p in evolution:
    sim.system.switch.value = p[0]
    sim.advance(p[1])
    sim.stabilise()
    
# plot the system
sim.plot()

# Plotting of the Graphs
The following code-cell defines some helper functions that create a [Plotly](https://plot.ly/) graph.   
The functions themselves are not particularly exciting, but have a look if you're interested ;-)

In [8]:
import plotly
import plotly.graph_objs as go
plotly.offline.init_notebook_mode(connected=True)  # don't talk to the plotly server, plot locally within a jupyter notebook


def plot_ports(sim, ytitle, ports, measurements=None):
    """
    This function takes a simulator and a list of ports, and plots the traces of these ports.
    Optionally, it will add o
    """
    store = sim.traces.datastore  # get the trace-datastore from the simulator; the datastore is a mapping that stores ports and their time-value traces and entities and their time-current states traces
    
    # create the lines we want to plot
    lines = []

    # create port traces
    for port in ports:
        lines.append(
            go.Scatter(x=[t[0] for t in store[port]], y=[t[1] for t in store[port]], 
                   name=f"{port._name} ({port.resource.unit})", # append the unit to the name in the legend
                   mode="markers+lines", marker={'symbol': 'x', 'size': 5}, line={'width': 1}  
                )
        )
    
    if measurements is not None:
        lines.append(
            go.Scatter(x=measurements[0], y=measurements[1], 
                       name=measurements[2], # append the unit to the name in the legend
                       mode="markers+lines", marker={'symbol': 'x', 'size': 5}, line={'width': 1}  
        ))
        
    # create a figure object
    fig = go.Figure(
        data=lines,
        layout=go.Layout(
            showlegend=True, 
            xaxis=dict(title='time (seconds)'),
            yaxis=dict(title=ytitle)
        )
    )
    
    # plot it!
    plotly.offline.iplot(fig)

def plot_sim(sim, volume_data=None):
    """
    Creates three plots:
        - Switch state over time
        - Flow measures over time
        - Volume measures over time
    """
    plot_ports(sim, "switch (on/off)", [sim.system.switch])
    plot_ports(sim, "flow (litres/sec)", [sim.system.tank.inflow, sim.system.tank.outflow])
    plot_ports(sim, "volume (litres)", [sim.system.tank.volume, sim.system.tank.spill, sim.system.tank.quantity, sim.system.tank.capacity], volume_data)

Here we create three plots of the simulation.  
One for the switch state, one for the flow rates and one for the volume simulation.

Note, that the last graph also displays the actual volume measurements of the system test that we created.  
As you can see, for the first 100 seconds the simulation is quite on point!  

Unfortunately, the outflow regression seems to be too eager and predicts a faster outflow rate than the system actually experiences.  
This results in a divergence for the phases without inflow. We should probably add more data for the machine learning training phase.

In [9]:
# add the system data from CSV and add it to the plot
from numpy import genfromtxt
measurements = genfromtxt("files/time_volume_test.csv", delimiter=',')
other_data = (measurements[:60,0], measurements[:60,1], "Volume measurements")  # this will be added to the volume graph
plot_sim(sim, other_data)